<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/Copy_of_Trainer_02_2e_5_1024_gamma4_12000_COPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q accelerate
!pip install -q huggingface_hub
!pip install -q scikit-learn

# transformers and datasets are Hugging Face libraries
!pip install -q transformers datasets

!pip install -q wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import json
import numpy as np
import os
import pandas as pd
import pickle
import sys
import time
import torch
import torch.nn.functional as F
import wandb

In [3]:
from datasets              import DatasetDict
from google.colab          import auth, drive, files, userdata
from huggingface_hub       import create_repo, login, upload_file, hf_hub_download
from huggingface_hub.utils import RepositoryNotFoundError
from sklearn.metrics       import accuracy_score, average_precision_score, classification_report, f1_score, precision_score, precision_recall_curve, precision_recall_fscore_support, recall_score, roc_auc_score
from torch.utils.data      import DataLoader
from tqdm.auto             import tqdm
from transformers          import AdamW, EvalPrediction, LongformerTokenizerFast, LongformerForSequenceClassification, Trainer, TrainingArguments
from torch.nn              import BCEWithLogitsLoss, Module

In [4]:
import os
from google.colab    import userdata
from huggingface_hub import login, hf_hub_download

In [5]:
# Hugging Face Authenticate
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")    # Store the key in os.environ
hf_token               = os.environ.get('HF_TOKEN')
login(token=hf_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [6]:
# Verify
!huggingface-cli whoami

claudelepere


In [7]:
file_path = hf_hub_download(
    repo_id   = "claudelepere/skill_classification",
    repo_type = "dataset",
    filename  = "test_model_eval_results.csv"
)
print(f"file_path: {file_path}")  # /root/.cache/huggingface/hub/datasets--claudelepere--skill_classification/snapshots/51ead81f69b1689fc19694b3f034585cde9f56e1/test_model_eval_results.csv

test_model_eval_results.csv:   0%|          | 0.00/313k [00:00<?, ?B/s]

file_path: /root/.cache/huggingface/hub/datasets--claudelepere--skill_classification/snapshots/e6293058f186e5445a37576053885a8fec8a8e02/test_model_eval_results.csv


Next, open in Colab or download to local

In [8]:
"""
# Check the Python version
print(sys.version)
print()

# Get the installed packages (you can see that conda is not installed (do not install it))
!pip list
print()

# Check system information
!cat /etc/os-release
!uname -m
print()

# Check the GPU details (only if the runtime type is T4 GPU)
#!nvidia-smi
#print()

# Check RAM
!free -h
print()

# Check disk space
!df -h
print()

# Get environment variables
for key, value in os.environ.items():
    print(f"{key}: {value}")
"""
!python -V

Python 3.11.11


In [9]:
print(f"currentdir: {os.getcwd()}")

currentdir: /content


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

device: cuda


In [11]:
datasetDict_zip_file_name = "dataset_11_12000.zip"
datasetDict_dir_name      = os.path.splitext(datasetDict_zip_file_name)[0]
print(f"datasetDict_zip_file_name: {datasetDict_zip_file_name}")
print(f"datasetDict_dir_name     : {datasetDict_dir_name}")
print()

datasetDict_zip_file_name: dataset_11_12000.zip
datasetDict_dir_name     : dataset_11_12000



In [12]:
# OOM: reduce batch size
#      small sizes (1 to 32):            PROs: better generalization in some cases
#                                        CONs: may produce noisier gradients
#      large sizes (128, 256, or higer): PROs: gradients are smoother, leading to more stable training
#                                        CONs: poorer generalization (overfitting) in some cases
#      intermediate sizes (32, 64):      combines the benefits of small and large sizes
batch_size = 8

In [13]:
# OOM: enable gradient accumulation to compensate for smaller batch sizes by accumulating gradients over several steps
#      effective batch size = per-device batch size x gradient accumulation steps;
#      in each iteration, the model computes the gradients, these gradients are immediately used to update the model parameters
gradient_accumulation_steps = 4  #<<<<<<<<<<<<<<<<<<< gradient_accumulation_steps may not be None => comment it in TrainingArguments

In [14]:
# OOM: use PYTORCH_CUDA_ALLOC_CONF to handle memory fragmentation
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [15]:
# OOM: check for zombie processes
if torch.cuda.is_available():
  !nvidia-smi
  torch.cuda.memory_summary()
!ps aux | grep python
!kill -9 <PID>
!nvidia-smi     # Checked if killed

Sun Jan 26 21:14:57 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              43W / 400W |      5MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [16]:
# OOM: use fp16 (half precision) mixed precision training
#      reduces memory requirements by up to 50%
fp16 = True

OOM: limit the number of GPU workers: 0 (default) or 1 in Colab
dataloader_num_workers = 1

In [17]:
# OOM: reduce model size or input tokens
#      1) LongformerTokenizer.from_pretrained('allenai/longformer-base/large-4096'): large/base: 435M/149M parameters
#      2) max_length: 4096 max for Longformer; 1 word can give several tokens, stop words are NOT discarded!
#         word_text_length_counts_sorted: jobs count                 : 50000
#                                         jobs count under  512 words: 44794  89.59%
#                                         jobs count under  640 words: 47894  95.79%
#                                         jobs count under  768 words: 49123  98.25%
#                                         jobs count under  896 words: 49691  99.38%
#                                         jobs count under 1024 words: 49917  99.83%
#                                         jobs count under 2048 words: 50000 100.00%
#                                         jobs count under 4096 words: 50000 100.00%
#max_length =  768    #      37 min    #
max_length = 1024    #      38 min    # GPU RAM: 12.2 / 40 GB
#max_length = 2048    # 1 hr 10 min    # GPU RAM: 21.4 / 40 GB
#max_length = 4096    # 2 hr 10 min    # GPU RAM: 39.5 / 40 GB => OutOfMemoryError

In [18]:
# OOM: free up GPU memory
torch.cuda.empty_cache()

In [19]:
# OOM: monitor GPU memory usage
!nividia-smi

/bin/bash: line 1: nividia-smi: command not found


In [20]:
# 1 epoch is a complete pass through the entire training dataset;
# with n datapoints and batch size = b, n/b iterations to complete 1 epoch;
# 1 iteration is a single update of the model's parameters
epochs = 5

In [21]:
# A common rule is to scale the learning rate proportionaly with the effective batch size
# note: get_linear_schedule_with_warmup <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
learning_rate = 2e-5  # 1e-5 x 32/8

Reduce the number of transformers layers
hidden_layers = 12    # 12 (default) or 6

In [22]:
# Threshold: 0.5 (default)
threshold = 0.2

In [23]:
if fp16:
  _fp = "fp16"
else:
  _fp = "fp32"

In [24]:
if 'gradient_accumulation_steps' not in globals():
  run_name = f"Longformer-multilabel-{datasetDict_dir_name}-length{max_length}-batch{batch_size}-epochs{epochs}-lr{learning_rate}-{_fp}-threshold{threshold}"
else:
  run_name = f"Longformer-multilabel-{datasetDict_dir_name}-length{max_length}-batch{batch_size}x{gradient_accumulation_steps}-epochs{epochs}-lr{learning_rate}-{_fp}-threshold{threshold}"
print(f"run_name                 : {run_name}")

run_name                 : Longformer-multilabel-dataset_11_12000-length1024-batch8x4-epochs5-lr2e-05-fp16-threshold0.2


In [25]:
def upload_unzip_dataset(file_name=datasetDict_zip_file_name):
  # Check if the file exists
  if not os.path.exists(file_name):
    print(f"'{file_name}' not found in /content. Uploading...")
    uploaded_files = files.upload()                              # Prompt file upload dialog
    if file_name not in uploaded_files:
      raise FileNotFoundError(f"'{file_name}' was not uploaded. Please try again.")
    print(f"'{file_name}' successfully uploaded to /content")
    uploaded_file_name = list(uploaded_files.keys())[0]          # Get the name of the uploaded file

    !unzip {uploaded_file_name}

    unzipped_dir_name = os.path.splitext(uploaded_file_name)[0]
    assert unzipped_dir_name==datasetDict_dir_name, "unzipped_dir_name != datasetDict_dir_name"
  else:
    print(f"'{datasetDict_dir_name}' already exists in /content.")

In [26]:
upload_unzip_dataset(datasetDict_zip_file_name)

'dataset_11_12000.zip' not found in /content. Uploading...


Saving dataset_11_12000.zip to dataset_11_12000.zip
'dataset_11_12000.zip' successfully uploaded to /content
Archive:  dataset_11_12000.zip
  inflating: dataset_11_12000/dataset_dict.json  
  inflating: dataset_11_12000/test/data-00000-of-00001.arrow  
  inflating: dataset_11_12000/test/dataset_info.json  
  inflating: dataset_11_12000/test/state.json  
  inflating: dataset_11_12000/train/data-00000-of-00001.arrow  
  inflating: dataset_11_12000/train/dataset_info.json  
  inflating: dataset_11_12000/train/state.json  
  inflating: dataset_11_12000/validation/data-00000-of-00001.arrow  
  inflating: dataset_11_12000/validation/dataset_info.json  
  inflating: dataset_11_12000/validation/state.json  


Hugging Face Authenticate

In [27]:
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")    # Store the key in os.environ
hf_token               = os.environ.get('HF_TOKEN')
login(token=hf_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [28]:
# Verify
!huggingface-cli whoami

claudelepere


Create the skill_classification repo on the Hugging Face Hub

In [29]:
HF_name         = "claudelepere/skill_classification"
repo_id_model   = HF_name
repo_id_dataset = HF_name

In [30]:
repo_model_url = create_repo(
    repo_id   = repo_id_model,
    repo_type = "model",
    private   = True,
    exist_ok  = True
)
print(f"Repo model url: {repo_model_url} created successfully as a private repo.")

Repo model url: https://huggingface.co/claudelepere/skill_classification created successfully as a private repo.


In [31]:
repo_dataset_url = create_repo(
    repo_id   = repo_id_dataset,
    repo_type = "dataset",
    private   = True,
    exist_ok  = True
)
print(f"Repo datasets url: {repo_dataset_url} created successfully as a private repo.")

Repo datasets url: https://huggingface.co/datasets/claudelepere/skill_classification created successfully as a private repo.


In [32]:
repo_id_dataset = f"datasets/{HF_name}"

In [33]:
print(f"repo_id_model: {repo_id_model}")
print(f"repo_id_dataset: {repo_id_dataset}")

repo_id_model: claudelepere/skill_classification
repo_id_dataset: datasets/claudelepere/skill_classification


W&B initialization

In [34]:
os.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")        # Store the key in os.environ
wandb_api_key               = os.environ.get('WANDB_API_KEY')
wandb.login(key=wandb_api_key)

wandb: Currently logged in as: claudelepere (claudelepere-c-cile-cy). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [35]:
try:
  wandb.init(
      project = "skill_classification",
      name    = run_name,
      entity  = "claudelepere-c-cile-cy",
      config  = {
          "learning_rate": learning_rate,
          "epochs"       : 5,
          "batch_size"   : batch_size
      }
  )
except wandb.errors.CommError as err:
  print(f"CommError: {err}")
except Exception as exc:
  print(f"Exception: {exc}")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create the dataset: 3 Hugging Face Dataset in a Hugging Face DatasetDict

In [36]:
datasetDict = DatasetDict.load_from_disk(datasetDict_dir_name)

In [37]:
print(f"datasetDict: {type(datasetDict)} {datasetDict.shape}\n{datasetDict}")
print(f"datasetDict.keys(): {datasetDict.keys()}")
print(f"datasetDict['train']: {type(datasetDict['train'])} {datasetDict['train'].shape}")
print(f"datasetDict['validation']: {type(datasetDict['validation'])} {datasetDict['validation'].shape}")
print(f"datasetDict['test']: {type(datasetDict['test'])} {datasetDict['test'].shape}")

datasetDict: <class 'datasets.dataset_dict.DatasetDict'> {'train': (9600, 8), 'validation': (1200, 8), 'test': (1200, 8)}
DatasetDict({
    train: Dataset({
        features: ['id', 'text', '390', '135', '136', '137', '138', '139'],
        num_rows: 9600
    })
    validation: Dataset({
        features: ['id', 'text', '390', '135', '136', '137', '138', '139'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['id', 'text', '390', '135', '136', '137', '138', '139'],
        num_rows: 1200
    })
})
datasetDict.keys(): dict_keys(['train', 'validation', 'test'])
datasetDict['train']: <class 'datasets.arrow_dataset.Dataset'> (9600, 8)
datasetDict['validation']: <class 'datasets.arrow_dataset.Dataset'> (1200, 8)
datasetDict['test']: <class 'datasets.arrow_dataset.Dataset'> (1200, 8)


In [38]:
example = datasetDict['train'][0]
print(f"datasetDict['train'][0]: {type(example)} {example.keys()}\n{example}")

datasetDict['train'][0]: <class 'dict'> dict_keys(['id', 'text', '390', '135', '136', '137', '138', '139'])
{'id': 153918, 'text': "Proximus SpearIT - Senior Web Security Engineer F5, Security, Firewall, LTM, ASM Proximus SpearIT Job description Install, deploy, manage and operate security solutions based on Web Application Firewall systems Mandatory knowledge of F5 Technologies (LTM and ASM) Good Knowledge of Firewall, Proxy, Networking (Routing & Switching) systems and technologies Testing of new configuration, equipment and releases Reporting on network/security usage and performance Participate to project and deployment of new security architecture Deliver technical solutions and improve the level of protection of the system Develop the technical documentation regarding the operational procedures Be part in the security incident response process and participate to On-Duty staffing outside business hours. Job requirements Advanced skills in F5 (LTM and ASM) architectures, solutions 

Create the label list and the id2label and label2id mappings.

In [39]:
"""
dataset 7_1000_125_125  ,  48 labels
dataset 7_128_18_54     ,  42 labels
dataset 8910_1087_68_204, 206 labels
dataset 11_1000         ,   6 labels
"""

'\ndataset 7_1000_125_125  ,  48 labels\ndataset 7_128_18_54     ,  42 labels\ndataset 8910_1087_68_204, 206 labels\ndataset 11_1000         ,   6 labels\n'

In [40]:
labels = [label for label in datasetDict['train'].features.keys() if label not in ['id', 'text']]
labels.sort()
print(f"labels: {type(labels)} {len(labels)}\n{labels}")
num_labels = len(labels)

labels: <class 'list'> 6
['135', '136', '137', '138', '139', '390']


In [41]:
id2label = {idx: label for idx, label in enumerate(labels)}
print(f"id2label: {type(id2label)} {len(id2label)}\n{id2label}")

id2label: <class 'dict'> 6
{0: '135', 1: '136', 2: '137', 3: '138', 4: '139', 5: '390'}


In [42]:
label2id = {label: idx for idx, label in enumerate(labels)}
print(f"label2id: {type(label2id)} {len(label2id)}\n{label2id}")

label2id: <class 'dict'> 6
{'135': 0, '136': 1, '137': 2, '138': 3, '139': 4, '390': 5}


Load tokenizer and model

In [43]:
model_name = "allenai/longformer-base-4096"

In [44]:
tokenizer = LongformerTokenizerFast.from_pretrained(model_name)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [45]:
model = LongformerForSequenceClassification.from_pretrained(
    model_name,
    num_labels   = num_labels,
    id2label     = id2label,
    label2id     = label2id,
    problem_type = 'multi_label_classification'
)

pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
# Configure attention window size
model.config.attention_window = 512

In [47]:
optimizer = AdamW(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Tokenize ('input_ids' and 'attention_mask'), add 'global_attention_mask' (for Longformer), add 'labels'

In [48]:
def preprocess_data(examples, indices):
  # Step 1: Extract text and tokenize
  text = examples['text']             # Batch of texts
  encoding = tokenizer(
      text,                           # Tokenize text
      truncation     = True,
      padding        = 'max_length',
      max_length     = max_length,
      return_tensors = 'pt'           # Return PyTorch tensors
  )

  # Step 2: Create and add the global attention mask
  global_attention_mask             = torch.zeros_like(encoding['input_ids'])  # Initialize global attention mask with zeros (same shape as input_ids)
  global_attention_mask[:, 0]       = 1                                        # Set global attention on the first token ([CLS], token ID=0) in each sequence
  encoding['global_attention_mask'] = global_attention_mask                    # Add the global_attention_mask to the batch

  # Step 3: Create and populate the label matrix
  labels_matrix = torch.zeros((len(text), len(labels)), dtype=torch.float32)   # Create an empty label matrix
  #print(f"labels_matrix: {type(labels_matrix)} {labels_matrix.shape}")
  #---------Populate label matrix
  for idx, label in enumerate(labels):
    #print(f"idx:{idx} label:{label}")
    if label in examples:
      labels_matrix[:, idx] = torch.tensor(
          [1.0 if val else 0.0 for val in examples[label]],
          dtype=torch.float32
          )
  print(f"labels_matrix: {type(labels_matrix)} {labels_matrix.shape}")

  encoding['labels'] = labels_matrix                                           # Add labels to the encoding
  print(f"encoding['labels']: {type(encoding['labels'])} {encoding['labels'].shape}")

  # encoding: <class 'transformers.tokenization_utils_base.BatchEncoding'> dict_keys(['input_ids', 'attention_mask', 'global_attention_mask', 'labels'])
  #   'input_ids': tensor([[
  #   'attention_mask': tensor([[
  #   'global_attention_mask': tensor([[
  #   'labels': tensor([[
  #print(f"1 preprocess_data call: encoding: {type(encoding)} {encoding.keys()}")

  return encoding

Create the 3 encoded datasets, train, validation and test

In [49]:
encoded_dataset = datasetDict.map(
    preprocess_data,
    batched        = True,
    remove_columns = datasetDict['train'].column_names,
    with_indices   = True
)
#train_dataset      = encoded_dataset['train']
#validation_dataset = encoded_dataset['validation']
#test_dataset       = encoded_dataset['test']

Map:   0%|          | 0/9600 [00:00<?, ? examples/s]

labels_matrix: <class 'torch.Tensor'> torch.Size([1000, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([1000, 6])
labels_matrix: <class 'torch.Tensor'> torch.Size([1000, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([1000, 6])
labels_matrix: <class 'torch.Tensor'> torch.Size([1000, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([1000, 6])
labels_matrix: <class 'torch.Tensor'> torch.Size([1000, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([1000, 6])
labels_matrix: <class 'torch.Tensor'> torch.Size([1000, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([1000, 6])
labels_matrix: <class 'torch.Tensor'> torch.Size([1000, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([1000, 6])
labels_matrix: <class 'torch.Tensor'> torch.Size([1000, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([1000, 6])
labels_matrix: <class 'torch.Tensor'> torch.Size([1000, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([1000, 6])


Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

labels_matrix: <class 'torch.Tensor'> torch.Size([1000, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([1000, 6])
labels_matrix: <class 'torch.Tensor'> torch.Size([200, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([200, 6])


Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

labels_matrix: <class 'torch.Tensor'> torch.Size([1000, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([1000, 6])
labels_matrix: <class 'torch.Tensor'> torch.Size([200, 6])
encoding['labels']: <class 'torch.Tensor'> torch.Size([200, 6])


train_labels_list_of_lists = train_dataset['labels'].tolist()
print("=============================================")
print(f"encoded_dataset: {type(encoded_dataset)} {encoded_dataset.shape}\n{encoded_dataset}")
print(f"train_dataset: {type(train_dataset)} {train_dataset.shape} {train_dataset.features}\n{train_dataset}")
print(f"validation_dataset: {type(validation_dataset)} {validation_dataset.shape} {validation_dataset.features}")
print(f"test_dataset: {type(test_dataset)} {test_dataset.shape} {test_dataset.features}")
print("---")
print(f"train_dataset['labels']: {type(train_dataset['labels'])} len={len(train_dataset['labels'])}\n{train_dataset['labels']}")
print("---")
print(f"train_dataset[0]['input_ids']: {type(train_dataset[0]['input_ids'])} {len(train_dataset[0]['input_ids'])}\n{train_dataset['input_ids'][0]}")
print(f"train_dataset[0]['attention_mask']: {type(train_dataset[0]['attention_mask'])} {len(train_dataset[0]['attention_mask'])}\n{train_dataset['attention_mask'][0]}")
print(f"train_dataset[0]['global_attention_mask']: {type(train_dataset[0]['global_attention_mask'])} {len(train_dataset[0]['global_attention_mask'])}\n{train_dataset['global_attention_mask'][0]}")
print(f"train_dataset[0]['labels']: {type(train_dataset[0]['labels'])} {len(train_dataset[0]['labels'])} {train_dataset[0]['labels']}")
print(f"train_dataset['labels'][0]: {type(train_dataset['labels'][0])} {len(train_dataset['labels'][0])}\n{train_dataset['labels'][0]}")

In [50]:
encoded_dataset.set_format('torch')
train_dataset      = encoded_dataset['train']
validation_dataset = encoded_dataset['validation']
test_dataset       = encoded_dataset['test']
print(f"train_dataset_tensor: {type(train_dataset)} {train_dataset.shape} {train_dataset.features}\n{train_dataset}")
print(f"train_dataset_tensor['input_ids']:             {type(train_dataset['input_ids'])}             len={len(train_dataset['input_ids'])}             shape={train_dataset['input_ids'].shape}            ") #\n{train_dataset['input_ids']}")
print(f"train_dataset_tensor['attention_mask']:        {type(train_dataset['attention_mask'])}        len={len(train_dataset['attention_mask'])}        shape={train_dataset['attention_mask'].shape}       ") #\n{train_dataset['attention_mask']}")
print(f"train_dataset_tensor['global_attention_mask']: {type(train_dataset['global_attention_mask'])} len={len(train_dataset['global_attention_mask'])} shape={train_dataset['global_attention_mask'].shape}") #\n{train_dataset['global_attention_mask']}")
print(f"train_dataset_tensor['labels']:                {type(train_dataset['labels'])}                len={len(train_dataset['labels'])}                shape={train_dataset['labels'].shape}               ") #\n{train_dataset['labels']}")

train_dataset_tensor: <class 'datasets.arrow_dataset.Dataset'> (9600, 4) {'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'global_attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None)}
Dataset({
    features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
    num_rows: 9600
})
train_dataset_tensor['input_ids']:             <class 'torch.Tensor'>             len=9600             shape=torch.Size([9600, 1024])            
train_dataset_tensor['attention_mask']:        <class 'torch.Tensor'>        len=9600        shape=torch.Size([9600, 1024])       
train_dataset_tensor['global_attention_mask']: <class 'torch.Tensor'> len=9600 shape=torch.Size([9600, 1024])
train_dataset_tensor['labels']:                <class 'torch.Tensor'>             

Truncated part

In [51]:
def get_truncated_part(text):
  tokens = tokenizer(
      text,
      truncation                = True,
      padding                   = 'max_length',
      max_length                = max_length,
      return_overflowing_tokens = True,
      return_tensors            = None
      )
  print(f"tokens.keys(): {tokens.keys()}")

  # Get the truncated tokens
  truncated_ids = tokens["input_ids"][0]
  print(f"truncated_ids: {type(truncated_ids)} {truncated_ids}")
  #overflow_ids  = tokens["overflow_to_sample_mapping"][0]
  #print(f"overflow_ids: {type(overflow_ids)} {overflow_ids}")

  # Decode the tokens back to text
  truncated_text = tokenizer.decode(truncated_ids, skip_special_tokens=True)
  #overflow_text  = tokenizer.decode(overflow_ids, skip_special_tokens=True)

  print(f"original_text :\n{text}")
  print(f"truncated_text:\n{truncated_text}")
  #print(f"overflow_text:\n{overflow_text}")

  original_tokens  = tokenizer.tokenize(text)
  truncated_tokens = tokenizer.tokenize(truncated_text)
  #overflow_tokens  = tokenizer.tokenize(overflow_text)

  print(f"original_tokens count : {len(original_tokens)}")
  print(f"truncated_tokens count: {len(truncated_tokens)}")
  #print(f"overflow_tokens count: {len(overflow_tokens)}")

In [52]:
example_text = datasetDict['train'][0]['text']
#get_truncated_part(example_text)

In [53]:
inputs = tokenizer(
    example_text,
    truncation     = True,
    padding        = 'max_length',
    max_length     = max_length,
    return_tensors = 'pt'
)

inputs: <class 'transformers.tokenization_utils_base.BatchEncoding'> dict_keys(['input_ids', 'attention_mask'])
  {'input_ids': tensor([[
  'attention_mask': tensor([[
print(f"inputs: {type(inputs)} {inputs.keys()}") #\n{inputs}")
print(f"inputs_ids: {type(inputs.input_ids)} {inputs.input_ids.shape}\n{inputs.input_ids}")
print(f"attention_mask: {type(inputs.attention_mask)} {inputs.attention_mask.shape}\n{inputs.attention_mask}")
print(f"labels: {inputs.labels.shape}")

4. Forward pass for multi-label classification

In [54]:
outputs = model(
    input_ids      = inputs.input_ids,
    attention_mask = inputs.attention_mask
    )

Initializing global attention on CLS token...


In [55]:
print(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")

outputs: <class 'transformers.models.longformer.modeling_longformer.LongformerSequenceClassifierOutput'> odict_keys(['logits'])
LongformerSequenceClassifierOutput(loss=None, logits=tensor([[ 0.1248, -0.1345,  0.0480, -0.2857, -0.0269, -0.0501]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None, global_attentions=None)


In [56]:
# Logits (= raw model outputs)
logits = outputs.logits
print(f"logits: {type(logits)} {logits.shape}\n{logits}")

logits: <class 'torch.Tensor'> torch.Size([1, 6])
tensor([[ 0.1248, -0.1345,  0.0480, -0.2857, -0.0269, -0.0501]],
       grad_fn=<AddmmBackward0>)


In [57]:
# Convert logits to probabilities
sigmoid = torch.nn.Sigmoid()
probs   = sigmoid(logits)
print(f"probs: {type(probs)} {probs.shape}\n{probs}")

probs: <class 'torch.Tensor'> torch.Size([1, 6])
tensor([[0.5312, 0.4664, 0.5120, 0.4291, 0.4933, 0.4875]],
       grad_fn=<SigmoidBackward0>)


In [58]:
example = encoded_dataset['train'][0]

In [59]:
print(f"example: {type(example)} {example.keys()}\n{example}")
print()
#print(f"example['input_ids']: {type(example['input_ids'])} {len(example['input_ids'])}\n{example['input_ids']}")
#print(f"example['attention_mask']: {type(example['attention_mask'])} {len(example['attention_mask'])}\n{example['attention_mask']}")
#print(f"example['labels']:  {type(example['labels'])} {len(example['labels'])}\n{example['labels']}")

example: <class 'dict'> dict_keys(['input_ids', 'attention_mask', 'global_attention_mask', 'labels'])
{'input_ids': tensor([    0, 10653,  1178,  ...,     1,     1,     1]), 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0]), 'global_attention_mask': tensor([1, 0, 0,  ..., 0, 0, 0]), 'labels': tensor([0., 0., 1., 1., 0., 0.])}



In [60]:
tokenizer.decode(example['input_ids'])

"<s>Proximus SpearIT - Senior Web Security Engineer F5, Security, Firewall, LTM, ASM Proximus SpearIT Job description Install, deploy, manage and operate security solutions based on Web Application Firewall systems Mandatory knowledge of F5 Technologies (LTM and ASM) Good Knowledge of Firewall, Proxy, Networking (Routing & Switching) systems and technologies Testing of new configuration, equipment and releases Reporting on network/security usage and performance Participate to project and deployment of new security architecture Deliver technical solutions and improve the level of protection of the system Develop the technical documentation regarding the operational procedures Be part in the security incident response process and participate to On-Duty staffing outside business hours. Job requirements Advanced skills in F5 (LTM and ASM) architectures, solutions and products Proven professional experience of several years (>4 years) in a similar operational environment Master's degree in 

In [61]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['137', '138']

Set PyTorch format to ensures correctness and compatibility with PyTorch pipelines

In [62]:
# The 3 Hugging Face Dataset are formatted as PyTorch Dataset
encoded_dataset.set_format('torch')

In [63]:
batch_size  = batch_size
metric_name = "f1"

In [64]:
training_args = TrainingArguments(
    output_dir                  = './training_results',  # where model predictions and checkpoints will be written during training
    overwrite_output_dir        = True,
    logging_dir                 = './logs',
    logging_steps               = 50,
    save_steps                  = 500,
    save_total_limit            = 2,
    eval_strategy               = 'epoch',
    save_strategy               = 'epoch',
    learning_rate               = learning_rate,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size  = batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,
    num_train_epochs            = epochs,
    weight_decay                = 0.01,
    load_best_model_at_end      = True,
    metric_for_best_model       = metric_name,
    run_name                    = run_name,
    fp16                        = fp16,
    #dataloader_num_workers      = dataloader_num_workers,
    report_to                  = 'wandb'
    )

Metrics
  source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/

In [65]:
def multi_label_metrics(logits, true_labels):
  """
  logits => sigmoid => probabilities => predictions

  Args:
    logits     : raw, unnormalized scores outputted by the model  (numpy array of shape (batch_size, num_labels))
    true_labels: actual labels for the data                       (numpy array of shape (batch_size, num_labels))

  Returns:
    metrics: dictionary of scores
  """
  average = 'micro'    # 'micro' or 'weighted'

  sigmoid = torch.nn.Sigmoid()
  probs   = sigmoid(torch.Tensor(logits))
  # next, use threshold to turn them into integer predictions
  preds = np.zeros(probs.shape)
  preds[np.where(probs > threshold)] = 1

  # compute metrics
  f1                   = f1_score               (y_true=true_labels, y_pred=preds,  average=average)    #, zero_division=1)
  precision            = precision_score        (y_true=true_labels, y_pred=preds,  average=average)    #, zero_division=1)
  recall               = recall_score           (y_true=true_labels, y_pred=preds,  average=average)    #, zero_division=1)
  roc_auc              = roc_auc_score          (y_true=true_labels, y_score=probs, average=average)
  precision_recall_auc = average_precision_score(y_true=true_labels, y_score=probs, average=average)
  accuracy             = accuracy_score         (y_true=true_labels, y_pred=preds)

  metrics = {

      'f1'                  : f1,
      'precision'           : precision,
      'recall'              : recall,
      'roc_auc'             : roc_auc,
      'precision_recall_auc': precision_recall_auc,
      'accuracy'            : accuracy
  }

  return metrics

In [66]:
def compute_metrics(p: EvalPrediction):
  preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
  result = multi_label_metrics(logits=preds,true_labels=p.label_ids)   # true_labels=p.label_ids<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

  return result

In [67]:
"""Let's verify a batch as well as a forward pass:"""

"Let's verify a batch as well as a forward pass:"

In [68]:
print(f"input_ids:              {type(encoded_dataset['train']['input_ids'][0])}\t{encoded_dataset['train']['input_ids'][0].shape}")
print(f"attention_mask:         {type(encoded_dataset['train']['attention_mask'][0])}\t{encoded_dataset['train']['attention_mask'][0].shape}")
print(f"global_attention_mask:  {type(encoded_dataset['train']['global_attention_mask'][0])}\t{encoded_dataset['train']['global_attention_mask'][0].shape}")
print(f"labels:                 {type(encoded_dataset['train'][0]['labels'])}\t{encoded_dataset['train'][0]['labels'].shape}")

input_ids:              <class 'torch.Tensor'>	torch.Size([1024])
attention_mask:         <class 'torch.Tensor'>	torch.Size([1024])
global_attention_mask:  <class 'torch.Tensor'>	torch.Size([1024])
labels:                 <class 'torch.Tensor'>	torch.Size([6])


Execute a forward pass for debugging or verification purposes (cf. BERT_3_1 in Notion BERT database)

In [69]:
outputs = model(
    input_ids      = encoded_dataset['train']['input_ids'][0].unsqueeze(0),
    attention_mask = encoded_dataset['train']['attention_mask'][0].unsqueeze(0),
    labels         = encoded_dataset['train'][0]['labels'].unsqueeze(0)
    )

In [70]:
print(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")

outputs: <class 'transformers.models.longformer.modeling_longformer.LongformerSequenceClassifierOutput'> odict_keys(['loss', 'logits'])
LongformerSequenceClassifierOutput(loss=tensor(0.7082, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.1248, -0.1345,  0.0480, -0.2857, -0.0269, -0.0501]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None, global_attentions=None)


In [71]:
"""# Define the weighted loss function

class_weights = torch.tensor([7.68, 2.15, 0.61, 0.47, 0.68, 6.26], dtype=torch.float32).to(device)
loss_fn       = BCEWithLogitsLoss(pos_weight=class_weights)  # For multi-label classification (binary classification per label)

## Class supports, class weigths, weighted loss function

Reminder:
*   df_jobs      : <class 'pandas.core.frame.DataFrame'>
*   df_jobs['id']: <class 'pandas.core.series.Series'>

dataset = Dataset.from_pandas(df_jobs)
*   dataset      : <class 'datasets.arrow_dataset.Dataset'>
*   dataset['id']: <class 'list'>

*   dataset_dict_jobs : <class 'datasets.dataset_dict.DatasetDict'>
*   train_dataset     : <class 'datasets.arrow_dataset.Dataset'>
*   validation_dataset: <class 'datasets.arrow_dataset.Dataset'>
*   test_dataset      : <class 'datasets.arrow_dataset.Dataset'>


We calculate the class supports for the train, validation and test datasets; the class weights and the weighted loss function are used for training only; the class supports of validation_dataset and test_dataset are calculated for information only.

def get_train_class_weights(datasetDict, labels):
  print(f"datasetDict: {type(datasetDict)} shape={datasetDict.shape}\n{datasetDict}")
  print(f"labels: {type(labels)} len={len(labels)}\n{labels}")

  dataset_train      = datasetDict['train']
  dataset_validation = datasetDict['validation']
  dataset_test       = datasetDict['test']

  def calculate_class_supports(dataset, labels):
    class_supports = dataset.map(
        lambda example: {col: example[col] for col in labels},
        batched=True
    ).to_pandas()[labels].sum(axis=0)
    return class_supports

  class_supports = {}

  for split_name, split_dataset in datasetDict.items():
    class_supports[split_name] = calculate_class_supports(split_dataset, labels)

  for split_name, split_class_supports in class_supports.items():
    print(f"{split_name}: {type(split_class_supports)} len={len(split_class_supports)}\n{split_class_supports}")

  train_class_supports_list = class_supports['train'].tolist()
  print(f"train_class_supports_list: {type(train_class_supports_list)} len={len(train_class_supports_list)} {train_class_supports_list}")

  train_class_supports_tensor = torch.tensor(train_class_supports_list, dtype=torch.float32)
  print(f"train_class_supports_tensor: {type(train_class_supports_tensor)} len={len(train_class_supports_tensor)} {train_class_supports_tensor}")

  train_total_samples = dataset_train.num_rows
  print(f"train_total_samples: {train_total_samples}")

  number_of_classes = len(labels)
  print(f"number_of_classes: {number_of_classes}")

  train_class_weights = train_total_samples / (number_of_classes * train_class_supports_tensor)
  print(f"train_class_weights: {type(train_class_weights)} len={len(train_class_weights)} {train_class_weights}")

  train_class_weights_sum = train_class_weights.sum()
  print(f"train_class_weights_sum: {train_class_weights_sum}")

  normalized_train_class_weights = (train_class_weights / train_class_weights_sum) * number_of_classes
  print(f"normalized_train_class_weights: {type(normalized_train_class_weights)} len={len(normalized_train_class_weights)} {normalized_train_class_weights}")

  # Positives samples per label
  supports = train_class_supports_tensor
  print(f"supports: {type(supports)} {len(supports)} {supports}")

  # Negatives samples per label
  negatives = train_total_samples - supports
  print(f"negatives: {type(negatives)} {len(negatives)} {negatives}")

  # pos_weights = negative to positive ratios
  pos_weights = negatives/supports
  print(f"pos_weights: {type(pos_weights)} {len(pos_weights)} {pos_weights}")

  # Normalize using min-max scaling
  normalized_pos_weights_minmax = (pos_weights - pos_weights.min()) / (pos_weights.max() - pos_weights.min())
  print(f"normalized_pos_weights_minmax: {type(normalized_pos_weights_minmax)} {len(normalized_pos_weights_minmax)} {normalized_pos_weights_minmax}")

  # Normalize using z-score standardization
  normalized_pos_weights_zscore = (pos_weights - pos_weights.mean()) / pos_weights.std()
  print(f"normalized_pos_weights_zscore: {type(normalized_pos_weights_zscore)} {len(normalized_pos_weights_zscore)} {normalized_pos_weights_zscore}")

  # Normalize using min-max scaling
  normalized_pos_weights_minmax = (pos_weights - pos_weights.min()) / (pos_weights.max() - pos_weights.min())
  print(f"normalized_pos_weights_minmax: {type(normalized_pos_weights_minmax)} {len(normalized_pos_weights_minmax)} {normalized_pos_weights_minmax}")

  # Normalize using z-score standardization
  normalized_pos_weights_zscore = (pos_weights - pos_weights.mean()) / pos_weights.std()
  print(f"normalized_pos_weights_zscore: {type(normalized_pos_weights_zscore)} {len(normalized_pos_weights_zscore)} {normalized_pos_weights_zscore}")

  # Normalize using sum-to-one
  normalized_pos_weights_sum1 = pos_weights / pos_weights.sum()
  print(f"normalized_pos_weights_sum1: {type(normalized_pos_weights_sum1)} {len(normalized_pos_weights_sum1)} {normalized_pos_weights_sum1}")

  return normalized_pos_weights_minmax
  #return normalized_pos_weights_zscore
  #return normalized_pos_weights_sum1

pos_weights = get_train_class_weights(datasetDict, labels)

loss_fn = BCEWithLogitsLoss(pos_weight=pos_weights.to(device))  # For multi-label classification (binary classification per label)
print(f"loss_fn: {type(loss_fn)} {loss_fn}")
"""

'# Define the weighted loss function\n\nclass_weights = torch.tensor([7.68, 2.15, 0.61, 0.47, 0.68, 6.26], dtype=torch.float32).to(device)\nloss_fn       = BCEWithLogitsLoss(pos_weight=class_weights)  # For multi-label classification (binary classification per label)\n\n## Class supports, class weigths, weighted loss function\n\nReminder:\n*   df_jobs      : <class \'pandas.core.frame.DataFrame\'>\n*   df_jobs[\'id\']: <class \'pandas.core.series.Series\'>\n\ndataset = Dataset.from_pandas(df_jobs)\n*   dataset      : <class \'datasets.arrow_dataset.Dataset\'>\n*   dataset[\'id\']: <class \'list\'>\n\n*   dataset_dict_jobs : <class \'datasets.dataset_dict.DatasetDict\'>\n*   train_dataset     : <class \'datasets.arrow_dataset.Dataset\'>\n*   validation_dataset: <class \'datasets.arrow_dataset.Dataset\'>\n*   test_dataset      : <class \'datasets.arrow_dataset.Dataset\'>\n\n\nWe calculate the class supports for the train, validation and test datasets; the class weights and the weighted l

In [72]:
def get_class_weights(labels=encoded_dataset['train']['labels']):
  print(f"labels: {type(labels)} len={len(labels)} shape={labels.shape}\n{labels}")

  num_samples, num_labels = labels.shape
  print(f"num_samples: {type(num_samples)} {num_samples}")
  print(f"num_labels:  {type(num_labels)}  {num_labels}")

  class_counts = labels.sum(dim=0)
  print(f"class_counts: {type(class_counts)} len={len(class_counts)}\n{class_counts}")

  pos_weights = (num_samples-class_counts) / class_counts
  print(f"pos_weights: {type(pos_weights)} len={len(pos_weights)}\n{pos_weights}")

  normalized_pos_weights_minmax = (pos_weights - pos_weights.min()) / (pos_weights.max() - pos_weights.min())
  print(f"normalized_pos_weights_minmax: {type(normalized_pos_weights_minmax)} {len(normalized_pos_weights_minmax)} {normalized_pos_weights_minmax}")

  normalized_pos_weights_zscore = (pos_weights - pos_weights.mean()) / pos_weights.std()
  print(f"normalized_pos_weights_zscore: {type(normalized_pos_weights_zscore)} {len(normalized_pos_weights_zscore)} {normalized_pos_weights_zscore}")

  normalized_pos_weights_sum1 = pos_weights / pos_weights.sum()
  print(f"normalized_pos_weights_sum1: {type(normalized_pos_weights_sum1)} {len(normalized_pos_weights_sum1)} {normalized_pos_weights_sum1}")

  #return pos_weights
  #return normalized_pos_weights_minmax
  #return normalized_pos_weights_zscore
  return normalized_pos_weights_sum1

In [73]:
pos_weights = get_class_weights()

labels: <class 'torch.Tensor'> len=9600 shape=torch.Size([9600, 6])
tensor([[0., 0., 1., 1., 0., 0.],
        [1., 1., 0., 0., 0., 1.],
        [0., 1., 1., 1., 0., 0.],
        ...,
        [0., 0., 1., 1., 1., 0.],
        [0., 0., 1., 1., 1., 0.],
        [0., 0., 0., 1., 1., 0.]])
num_samples: <class 'int'> 9600
num_labels:  <class 'int'>  6
class_counts: <class 'torch.Tensor'> len=6
tensor([ 554., 1789., 6639., 8754., 6055.,  715.])
pos_weights: <class 'torch.Tensor'> len=6
tensor([16.3285,  4.3661,  0.4460,  0.0966,  0.5855, 12.4266])
normalized_pos_weights_minmax: <class 'torch.Tensor'> 6 tensor([1.0000, 0.2630, 0.0215, 0.0000, 0.0301, 0.7596])
normalized_pos_weights_zscore: <class 'torch.Tensor'> 6 tensor([ 1.5167, -0.1917, -0.7515, -0.8014, -0.7316,  0.9595])
normalized_pos_weights_sum1: <class 'torch.Tensor'> 6 tensor([0.4768, 0.1275, 0.0130, 0.0028, 0.0171, 0.3628])


In [74]:
loss_fn = BCEWithLogitsLoss(pos_weight=pos_weights.to(device))

raise Exception("Stop here")

In [75]:
class FocalLoss(Module):
  """
  Focal Loss implementation
  """
  def __init__(self, alpha=1.0, gamma=2.0, logits=False, reduce=True):
    super(FocalLoss, self).__init__()
    self.alpha   = alpha
    self.gamma   = gamma
    self.logits  = logits  # This flag is to indicate whether input is logits or probability
    self.reduce  = reduce

  # inputs  = model's predictions: PyTorch tensor, shape=(batch_size, num_classes)
  # targets = ground truth labels: PyTorch tensor, shape=same as inputs shape
  def forward(self, inputs, targets):
    # Here, we check if input is probability or logits
    if self.logits:
      # Input is logits
      BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
    else:
      # Input is probability
      BCE_loss = F.binary_cross_entropy(inputs, targets, reduction='none')
    pt = torch.exp(-BCE_loss)
    F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

    if self.reduce:
      return torch.mean(F_loss)
    else:
      return F_loss

  def __repr__(self):
    return f"FocalLoss(alpha={self.alpha}, gamma={self.gamma}, logits={self.logits}, reduce={self.reduce})"

  def __str__(self):
    return f"FocalLoss(alpha={self.alpha}, gamma={self.gamma}, logits={self.logits}, reduce={self.reduce})"

  def __call__(self, inputs, targets):
    return self.forward(inputs, targets)

In [76]:
focal_loss_fn = FocalLoss(alpha=0.5, gamma=4.0, logits=True, reduce=True)
print(f"focal_loss_fn: {type(focal_loss_fn)} {focal_loss_fn}")

focal_loss_fn: <class '__main__.FocalLoss'> FocalLoss(alpha=0.5, gamma=4.0, logits=True, reduce=True)


CustomTrainer

In [77]:
class CustomTrainer(Trainer):

  def __init__(self, *args, loss_fn=None, **kwargs):
    super().__init__(*args, **kwargs)
    self.loss_fn = loss_fn

  """
  # No print in compute_loss because out of memory because prints are batch per batch
  def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):

    #print(f"inputs passed to compute_loss: {inputs.keys()}")
    #input_ids             = inputs['input_ids']                        # shape: batch_size, sequence_length
    #attention_mask        = inputs['attention_mask']                   # shape: batch_size, sequence_length
    #global_attention_mask = inputs.get('global_attention_mask', None)  # shape: batch_size, sequence_length; optional as LongFormer specific
    labels                = inputs.pop('labels', None)                 # shape: batch_size, num_labels; needed for loss computation, not required by the model

    #outputs = model(**inputs, global_attention_mask=global_attention_mask)  # Forward pass
    # Forward pass
    #outputs = model(
    #    input_ids             = input_ids,
    #    attention_mask        = attention_mask,
    #    global_attention_mask = global_attention_mask,
    #    labels                = labels
    #)
    outputs = model(**inputs, labels=labels)
    #print(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")
    logits = outputs.logits  # shape: (batch_size, num_labels)

    # If labels are provided, compute loss
    if labels is not None:
      # Use the custom loss function if provided
      if self.loss_fn is not None:
        loss = self.loss_fn(logits, labels)  # Compute weighted loss
      else:
        # Default loss: BCEWithLogitsLoss
        loss_fn = BCEWithLogitsLoss()
        loss    = loss_fn(logits, labels)    # Compute loss
      return (loss, outputs) if return_outputs else loss

    # If no labels, return outputs only, for evaluation or inference
    return outputs
    """
  def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
    labels  = inputs.pop('labels', None)
    outputs = model(**inputs, labels=labels)
    logits  = outputs.logits

    if labels is not None:
      if self.loss_fn is not None:
        loss = self.loss_fn(logits, labels)
      else:
        loss_fn = BCEWithLogitsLoss()
        loss    = loss_fn(logits, labels)
      return (loss, outputs) if return_outputs else loss

    return outputs

Create a Hugging Face's transformers trainer (which abstracts the training loop)

In [78]:
"""
trainer = CustomTrainer(
    model           = model,
    args            = training_args,
    train_dataset   = encoded_dataset["train"],
    eval_dataset    = encoded_dataset["validation"],
    compute_metrics = compute_metrics,                # Optional: custom metrics function
    loss_fn         = focal_loss_fn,
)
"""

trainer = Trainer(
    model           = model,
    args            = training_args,
    train_dataset   = encoded_dataset["train"],
    eval_dataset    = encoded_dataset["validation"],
    compute_metrics = compute_metrics,                # Optional: custom metrics function
)

Train

In [79]:
"""
predictions_output: <class 'transformers.trainer_utils.PredictionOutput'> len=3:
- predictions: np.ndarray          raw model outputs = logits
- label_ids  : np.ndarray or None: true labels corresponding to the predictions, if available
- metrics    : dict              : metrics computed during prediction step
"""
"""
predictions_output = trainer.predict(test_dataset)
print(f"predictions_output: {type(predictions_output)} len={len(predictions_output)}\n{predictions_output}")  # <class 'transformers.trainer_utils.PredictionOutput'> len=3
logits = predictions_output.predictions
print(f"logits: {type(logits)} len={len(logits)} shape={logits.shape}\n{logits}")


raise Exception("I'm here")
"""

'\npredictions_output = trainer.predict(test_dataset)\nprint(f"predictions_output: {type(predictions_output)} len={len(predictions_output)}\n{predictions_output}")  # <class \'transformers.trainer_utils.PredictionOutput\'> len=3\nlogits = predictions_output.predictions\nprint(f"logits: {type(logits)} len={len(logits)} shape={logits.shape}\n{logits}")\n\n\nraise Exception("I\'m here")\n'

In [80]:
trainer_train = trainer.train()
print(f"trainer_train: {type(trainer_train)} len={len(trainer_train)}\n{trainer_train}")

Epoch,Training Loss,Validation Loss,F1,Precision,Recall,Roc Auc,Precision Recall Auc,Accuracy
1,1.376300,0.346174,0.827337,0.738267,0.940848,0.923455,0.887738,0.286667
2,1.149100,0.306583,0.818849,0.717429,0.953664,0.940868,0.913973,0.271667
3,1.051000,0.306775,0.851683,0.790763,0.922774,0.942647,0.915109,0.426667
4,0.992100,0.294448,0.851216,0.779661,0.937233,0.946580,0.922567,0.416667
5,0.873900,0.293697,0.852498,0.780448,0.939205,0.947255,0.923164,0.417500


trainer_train: <class 'transformers.trainer_utils.TrainOutput'> len=3
TrainOutput(global_step=1500, training_loss=1.142166155497233, metrics={'train_runtime': 2402.715, 'train_samples_per_second': 19.977, 'train_steps_per_second': 0.624, 'total_flos': 3.1529784508416e+16, 'train_loss': 1.142166155497233, 'epoch': 5.0})


In [81]:
file_path = "trainer_train.json"
with open(file_path, "w") as f:
  json.dump(trainer_train, f)
print(f"Train output successfully saved to {file_path}.")

Train output successfully saved to trainer_train.json.


In [82]:
print("Training successfully completed.")

Training successfully completed.


In [83]:
"""## Upload model, tokenizer, train results, evaluate results"""

'## Upload model, tokenizer, train results, evaluate results'

Upload the tokenizer and the model to the Hugging Face Hub

In [84]:
# Push the tokenizer and the model
tokenizer.push_to_hub(repo_id_model)
model.push_to_hub(repo_id_model)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


model.safetensors:   0%|          | 0.00/595M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/claudelepere/skill_classification/commit/f39301fe8df8b6af03ed982874d0eec688d80e4a', commit_message='Upload LongformerForSequenceClassification', commit_description='', oid='f39301fe8df8b6af03ed982874d0eec688d80e4a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/claudelepere/skill_classification', endpoint='https://huggingface.co', repo_type='model', repo_id='claudelepere/skill_classification'), pr_revision=None, pr_num=None)

Verify the Upload

In [85]:
# Load the tokenizer and the model from the Hugging Face Hub
tokenizer = LongformerTokenizerFast.from_pretrained(repo_id_model)
model     = LongformerForSequenceClassification.from_pretrained(repo_id_model)

# Test the tokenizer and the model
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt", truncation=True, padding=True)
outputs = model(**inputs)
print(outputs)

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/595M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Input ids are automatically padded to be a multiple of `config.attention_window`: 512


LongformerSequenceClassifierOutput(loss=None, logits=tensor([[-1.8487,  0.4811,  2.8321,  1.3121, -1.5867, -1.3656]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None, global_attentions=None)


Upload Train results to HF repo_id_dataset

In [86]:
# Train
upload_file(
    path_or_fileobj = 'trainer_train.json',
    path_in_repo    = 'trainer_train.json',
    repo_id         = HF_name,
    repo_type       = 'dataset'
)

CommitInfo(commit_url='https://huggingface.co/datasets/claudelepere/skill_classification/commit/8a6fb7526affb53b47f9a1b735e1634cf47d8d09', commit_message='Upload trainer_train.json with huggingface_hub', commit_description='', oid='8a6fb7526affb53b47f9a1b735e1634cf47d8d09', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/claudelepere/skill_classification', endpoint='https://huggingface.co', repo_type='dataset', repo_id='claudelepere/skill_classification'), pr_revision=None, pr_num=None)

In [87]:
"""To Get Results of Evaluation and Test"""

'To Get Results of Evaluation and Test'

In [88]:
def get_results(model, dataset, batch_size, threshold, phase):
  # Clear GPU cache
  torch.cuda.empty_cache()

  # Set the model to evaluation mode to disable dropout and other training-specific behaviors
  model.eval()

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  dataLoader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

  all_preds       = []
  all_probs       = []
  all_true_labels = []

  for batch in tqdm(dataLoader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
      outputs = model(**batch)
    logits = outputs.logits

    # Convert logits to probabilities and probabilities to predictions
    sigmoid = torch.nn.Sigmoid()
    probs   = sigmoid(logits).cpu().numpy()    # Convert to Numpy
    preds   = (probs > threshold).astype(int)  # Convert to binary Numpy array; convert the boolean result to int (0 or 1)

    # Accumulate probabilities, predictions and labels
    all_probs.append(probs)
    all_preds.append(preds)
    all_true_labels.append(batch['labels'].cpu().numpy())

  # Concatenate results from all batches
  all_probs       = np.concatenate(all_probs, axis=0)        # shape: [num_samples, num_labels]
  all_preds       = np.concatenate(all_preds, axis=0)        # shape: [num_samples, num_labels]
  all_true_labels = np.concatenate(all_true_labels, axis=0)  # shape: [num_samples, num_labels]

  print(f"all_probs:       {type(all_probs)} {all_probs.shape}")
  print(f"all_preds:       {type(all_preds)} {all_preds.shape}")
  print(f"all_true_labels: {type(all_true_labels)} {all_true_labels.shape}")
  results_df = pd.DataFrame({
      'predictions'  : [list(pred)  for pred  in all_preds],
      'probabilities': [list(prob)  for prob  in all_probs],
      'true_labels'  : [list(label) for label in all_true_labels]
  })
  results_file_path = f"{phase}_results.csv"
  results_df.to_csv(results_file_path, index=False)

  # Classification report for precision, recall, F1 score
  report = classification_report(
      y_true        = all_true_labels,
      y_pred        = all_preds,
      target_names  = labels,
      zero_division = 0,
      output_dict   = True
  )
  print(f"Classification Report:\n{report}")

  # ROC AUC for multi-label classification
  roc_auc = roc_auc_score(
      y_true  = all_true_labels,
      y_score = all_probs,
      average = 'micro'
  )
  print(f"ROC AUC: {roc_auc}")

  metrics = {
      'classification_report': report,
      'roc_auc'              : roc_auc
  }
  metrics_file_path = f"{phase}_metrics.json"
  with open(metrics_file_path, "w") as f:
    json.dump(metrics, f, indent=4)

  print(f"{phase} Results Saved to {results_file_path}")
  print(f"{phase} Metrics Saved to {metrics_file_path}")

In [89]:
def get_results_with_threshold_tuning(model, dataset, batch_size, average, tune_thresholds=False, phase='eval'):
  """
  Evaluate a model on a given dataset and optionally tune thresholds for multi-label classification.

  Args:
    model          : the trained model to evaluate
    dataset        : the dataset to evaluate (validation or test)
    batch_size     : batch size for DataLoader
    tune_thresholds: whether to tune thresholds per label (default = False)
    average        : the averaging method for metrics ('micro', 'macro', 'weighted', etc.)
    phase          : the phase (eval or test) for saving results (default = 'eval')

  Returns:
    metrics: a dictionary with metrics and optionally tuned thresholds
  """

  # Clear GPU cache
  torch.cuda.empty_cache()

  # Set the model to evaluation mode to disable dropout and other training-specific behaviors
  model.eval()

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  # Create DataLoader
  dataLoader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

  all_preds       = []
  all_probs       = []
  all_true_labels = []

  for batch in tqdm(dataLoader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
      outputs = model(**batch)
    logits = outputs.logits

  # Convert logits to probabilities
  sigmoid = torch.nn.Sigmoid()
  probs   = sigmoid(logits).cpu().numpy()    # Convert to Numpy
  all_probs.append(probs)
  all_true_labels.append(batch['labels'].cpu().numpy())

  # Concatenate results from all batches
  all_probs       = np.concatenate(all_probs, axis=0)        # shape: [num_samples, num_labels]
  all_true_labels = np.concatenate(all_true_labels, axis=0)  # shape: [num_samples, num_labels]

  # Initialize thresholds (default = 0.5)
  thresholds = np.full(all_probs.shape[1], 0.5)  # shape: [num_samples]

  # Thresholds tuning (if enabled)
  if tune_thresholds:
    print(f"Tuning thresholds for {phase} phase")
    # Iterate over labels
    for label_idx in range(all_probs.shape[1]):
      precision, recall, thresholds_label = precision_recall_curve(all_true_labels[:, label_idx], all_probs[:, label_idx])
      f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)  # Avoid division by zero
      best_threshold_idx = np.argmax(f1_scores)
      # Set the best threshold for this label
      thresholds[label_idx] = thresholds_label[best_threshold_idx]

  # Apply thresholds to probabilities to generate predictions
  all_preds = (all_probs > thresholds).astype(int)  # Convert to binary Numpy array; convert the boolean result to int (0 or 1)

  # Save predictions, probabilities, and true labels to a DataFrame
  results_df = pd.DataFrame({
      'predictions'  : [list(pred)  for pred  in all_preds],
      'probabilities': [list(prob)  for prob  in all_probs],
      'true_labels'  : [list(label) for label in all_true_labels]
  })
  results_file_path = f"{phase}_results.csv"
  results_df.to_csv(results_file_path, index=False)

  # Compute metrics
  print(f"Computing metrics for {phase} phase")
  classification_report_dict = classification_report(
      y_true        = all_true_labels,
      y_pred        = all_preds,
      target_names  = labels,
      zero_division = 0,
      output_dict   = True
  )

  # Compute ROC auc
  roc_auc = roc_auc_score(
      y_true  = all_true_labels,
      y_score = all_probs,
      average = average
  )

  # Compute Precision-Recall auc
  precision_recall_auc = average_precision_score(all_true_labels, all_probs, average=average)

  #precision, recall, _ = precision_recall_curve(all_true_labels, all_probs)
  #precision_recall_auc = auc(recall, precision)

  metrics = {
      'classification_report': classification_report_dict,
      'roc_auc'              : roc_auc,
      'precision_recall_auc' : precision_recall_auc,
      'thresholds'           : thresholds.tolist() if tune_thresholds else "Default (0.5)",  # Convert numpy array to list
  }

  metrics_file_path = f"{phase}_metrics.json"
  with open(metrics_file_path, "w") as f:
    json.dump(metrics, f, indent=4)

  print(f"{phase} Results Saved to {results_file_path}")
  print(f"{phase} Metrics Saved to {metrics_file_path}")

  return metrics


In [90]:
def tune_thresholds(true_labels, probs, id2label):
  """
  Tune thresholds for each label to maximize F1 alone, as F1 balances precision and recall into a single metric.

  Args:
    true_labels: actual labels for the data                                      (numpy array of shape (num_samples, num_labels))
    probs      : predicted probabilities                                         (numpy array of shape (num_samples, num_labels))
    id2label   : dictionary mapping label indices (int) to label names (string)

  Returns:
    best_thresholds: best threshold for each label                                                      (numpy array of shape (num_labels,))
    best_metrics   : dictionary of best F1, precision_for_best_f1 and recall_for_best_f1 for each label (dictionary of numpy arrays)
  """
  thresholds      = np.linspace(0.1, 0.9, 9)
  best_thresholds = np.zeros(len(id2label))
  best_metrics    = {label: {'f1': 0.0, 'precision': 0.0, 'recall': 0.0} for label in id2label.values()}

  for label_idx, label in id2label.items():
    for threshold in thresholds:
      pred                     = (probs[:, label_idx] > threshold).astype(int)
      precision, recall, f1, _ = precision_recall_fscore_support(true_labels[:, label_idx], pred, average='binary', zero_division=0)
      if f1 > best_metrics[label]['f1']:
        best_thresholds[label_idx]       = threshold
        best_metrics[label]['f1']        = f1
        best_metrics[label]['precision'] = precision
        best_metrics[label]['recall']    = recall

  return best_thresholds, best_metrics


In [91]:
#def compute_metrics_with_threshold(eval_preds: EvalPrediction, thresholds, id2label):
def compute_metrics_with_threshold(probs, label_ids, thresholds, id2label):
  """
  Compute metrics during evaluation or test, by applying tuned thresholds
  """
  #logits  = eval_preds.predictions
  #labels  = eval_preds.label_ids
  #sigmoid = torch.nn.Sigmoid  # Sigmoid or numpy?
  #probs   = sigmoid(logits).cpu().numpy()
  preds   = np.zeros_like(probs)

  # Apply threshold per label
  for label_idx in id2label.keys():
    preds[:, label_idx] = (probs[:, label_idx] > thresholds[label_idx]).astype(int)

  # Compute metrics
  f1                    = f1_score               (label_ids, preds, average='micro')
  precision             = precision_score        (label_ids, preds, average='micro')
  recall                = recall_score           (label_ids, preds, average='micro')
  accuracy              = accuracy_score         (label_ids, preds)
  roc_auc               = roc_auc_score          (label_ids, probs, average='micro')
  precision_recall_auc  = average_precision_score(label_ids, probs, average='micro')

  # Use id2label for target_names
  report = classification_report(label_ids, preds, target_names=id2label.values(), zero_division=0)

  metrics = {
      'f1'                   : f1,
      'precision'            : precision,
      'recall'               : recall,
      'accuracy'             : accuracy,
      'roc_auc'              : roc_auc,
      'precision_recall_auc' : precision_recall_auc,
      'thresholds'           : thresholds.tolist(),
      'classification_report': report
  }

  return metrics


In [92]:
"""
def predict_with_thresholds(trainer, dataset, id2label, threshold_tuning=False, thresholds=None):

    Predicts using trainer.predict(), with optional threshold tuning.

    Parameters:
    - trainer         : Hugging Face Trainer or CustomTrainer instance
    - dataset         : dataset to predict on
    - id2label        : dictionary mapping label indices (int) to label names (string)
    - threshold_tuning: boolean to enable threshold tuning
    - thresholds      : custom thresholds for classification (optional)

    Returns:
    - predictions                               : final binary predictions
    - true_labels                               : ground truth labels from the dataset
    - best_thresholds (if threshold_tuning=True): optimized threshold for each label
    - best_metrics    (if threshold_tuning=True): dictionary of best F1, precision_for_best_f1 and recall_for_best_f1 for each label

    # Predict
    predictions_output = trainer.predict(dataset)        # <class 'transformers.trainer_utils.PredictionOutput'> len   = 3
    predictions        = predictions_output.predictions  # <class 'numpy.ndarray'>                               shape = (12, 6)
    label_ids          = predictions_output.label_ids    # <class 'numpy.ndarray'>                               shape = (12, 6)
    metrics            = predictions_output.metrics      # <class 'dict'                                         len   = 10

    print(f"predictions: {type(predictions)} {predictions.shape}")
    print(f"label_ids  : {type(label_ids)}  {label_ids.shape}")
    print(f"metrics    : {type(metrics)} len={len(metrics)}")

    logits             = predictions
    true_labels        = label_ids

    # Convert logits to probabilities
    probabilities = 1 / (1 + np.exp(-logits))            # <class 'numpy.ndarray'>                               shape = (12, 6)
    print(f"probabilities: {type(probabilities)} {probabilities.shape}")

    best_thresholds, best_metrics = tune_thresholds(true_labels, probabilities, id2label)
    print(f"best_thresholds: {type(best_thresholds)} {best_thresholds.shape}\n{best_thresholds}")
    print(f"best_metrics   : {type(best_metrics)} len={len(best_metrics)}\n{best_metrics}")

    metrics = compute_metrics_with_threshold(predictions, best_thresholds, id2label)

    return metrics
"""

'\ndef predict_with_thresholds(trainer, dataset, id2label, threshold_tuning=False, thresholds=None):\n    \n    Predicts using trainer.predict(), with optional threshold tuning.\n\n    Parameters:\n    - trainer         : Hugging Face Trainer or CustomTrainer instance\n    - dataset         : dataset to predict on\n    - id2label        : dictionary mapping label indices (int) to label names (string)\n    - threshold_tuning: boolean to enable threshold tuning\n    - thresholds      : custom thresholds for classification (optional)\n\n    Returns:\n    - predictions                               : final binary predictions\n    - true_labels                               : ground truth labels from the dataset\n    - best_thresholds (if threshold_tuning=True): optimized threshold for each label\n    - best_metrics    (if threshold_tuning=True): dictionary of best F1, precision_for_best_f1 and recall_for_best_f1 for each label\n    \n    # Predict\n    predictions_output = trainer.predict(

In [93]:
def predict_with_thresholds(trainer, dataset, id2label, threshold_tuning=False, thresholds=None):
    """
    Predicts using trainer.predict(), with optional thresholds tuning.

    Parameters:
      - trainer         : Hugging Face Trainer or CustomTrainer instance
      - dataset         : dataset to predict on
      - id2label        : dictionary mapping label indices (int) to label names (string)
      - threshold_tuning: boolean to enable threshold tuning
      - thresholds      : custom thresholds for classification (optional)

    Returns:
      - predictions                               : final binary predictions
      - true_labels                               : ground truth labels from the dataset
      - best_thresholds (if threshold_tuning=True): optimized threshold for each label
      - best_metrics    (if threshold_tuning=True): dictionary of best F1, precision_for_best_f1 and recall_for_best_f1 for each label TODO
    """
    # Predict
    predictions_output = trainer.predict(dataset)        # <class 'transformers.trainer_utils.PredictionOutput'> len   = 3
    predictions        = predictions_output.predictions  # <class 'numpy.ndarray'>                               shape = (12, 6) TODO
    label_ids          = predictions_output.label_ids    # <class 'numpy.ndarray'>                               shape = (12, 6) TODO
    metrics            = predictions_output.metrics      # <class 'dict'                                         len   = 10

    print(f"predictions: {type(predictions)} shape={predictions.shape}\n{predictions}")
    print(f"label_ids  : {type(label_ids)}   shape={label_ids.shape}  \n{label_ids}")
    print(f"metrics    : {type(metrics)}     len={len(metrics)}       \n{json.dumps(metrics, indent=4)}")

    logits             = predictions
    true_labels        = label_ids

    # Convert logits to probabilities
    probabilities = 1 / (1 + np.exp(-logits))            # <class 'numpy.ndarray'>                               shape = (12, 6) TODO
    print(f"probabilities: {type(probabilities)} shape={probabilities.shape}")

    best_thresholds, best_metrics = tune_thresholds(true_labels, probabilities, id2label)
    print(f"best_thresholds: {type(best_thresholds)} shape={best_thresholds.shape}\n{best_thresholds}")
    print(f"best_metrics   : {type(best_metrics)}    len={len(best_metrics)}\n{best_metrics}")

    # Create EvalPrediction object
    # eval_preds = EvalPrediction(predictions=predictions, label_ids=label_ids)
    #from transformers import EvalPrediction
    #eval_preds = EvalPrediction(predictions=probabilities, label_ids=label_ids)  # Pass probabilities instead of predictions

    #metrics = compute_metrics_with_threshold(eval_preds, best_thresholds, id2label)
    metrics = compute_metrics_with_threshold(probabilities, label_ids, best_thresholds, id2label)

    return metrics

In [94]:
metrics = predict_with_thresholds(trainer, validation_dataset, id2label, threshold_tuning=True, thresholds=None)
print(f"metrics: {type(metrics)} len={len(metrics)}\n{json.dumps(metrics, indent=4)}")
print(f"metrics['classification_report']: {type(metrics['classification_report'])} len={len(metrics['classification_report'])}\n{metrics['classification_report']}")


raise Exception("I'm here")

predictions: <class 'numpy.ndarray'> shape=(1200, 6)
[[-3.8144531   0.27075195  3.6210938   3.3828125   0.27807617 -3.0449219 ]
 [-5.1132812  -2.7578125   1.5566406   4.0078125   0.74902344 -4.6132812 ]
 [-4.5820312  -2.8398438   3.5722656   4.0976562   0.57421875 -4.3242188 ]
 ...
 [-4.1835938  -1.2373047   3.8417969   3.1503906  -0.7397461  -3.546875  ]
 [-4.9960938  -2.5878906   0.765625    4.21875     2.4707031  -4.4960938 ]
 [-4.671875   -1.4228516   2.6464844   3.4960938  -0.515625   -3.703125  ]]
label_ids  : <class 'numpy.ndarray'>   shape=(1200, 6)  
[[0. 0. 1. 1. 1. 0.]
 [0. 1. 1. 1. 0. 0.]
 [0. 0. 1. 1. 1. 0.]
 ...
 [0. 0. 1. 1. 1. 0.]
 [0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 1. 1. 0.]]
metrics    : <class 'dict'>     len=10       
{
    "test_loss": 0.29369696974754333,
    "test_f1": 0.8524981357196122,
    "test_precision": 0.7804478427089022,
    "test_recall": 0.9392047321721985,
    "test_roc_auc": 0.9472554440004392,
    "test_precision_recall_auc": 0.9231642246713727,
    "te

Exception: I'm here

In [ ]:
"""## Evaluate

After training, we evaluate our model on the validation set.
"""

First evaluate results

In [ ]:
phase_evaluate = 'evaluate_model_eval'

In [ ]:
get_results(
    model      = model,
    dataset    = validation_dataset,
    batch_size = batch_size,
    threshold  = threshold,
    phase      = phase_evaluate
)

In [ ]:
print("First evaluation successfully completed.")

Second evaluate results

In [ ]:
trainer_evaluate = trainer.evaluate()
print(f"trainer_evaluate: {type(trainer_evaluate)} len={len(trainer_evaluate)}\n{trainer_evaluate}")

In [ ]:
file_path = "trainer_evaluate.json"
with open(file_path, "w") as f:
  json.dump(trainer_evaluate, f)
print(f"Evaluate output successfully saved to {file_path}.")

In [ ]:
print("Second evaluation successfully completed.")

Upload Evaluate Results to HF repo_id_dataset

In [ ]:
upload_file(
    path_or_fileobj = f"{phase_evaluate}_results.csv",
    path_in_repo    = f"{phase_evaluate}_results.csv",
    repo_id         = HF_name,
    repo_type       = 'dataset'
)
upload_file(
    path_or_fileobj = f"{phase_evaluate}_metrics.json",
    path_in_repo    = f"{phase_evaluate}_metrics.json",
    repo_id         = HF_name,
    repo_type       = 'dataset'
)
upload_file(
    path_or_fileobj = 'trainer_evaluate.json',
    path_in_repo    = 'trainer_evaluate.json',
    repo_id         = HF_name,
    repo_type       = 'dataset'
)

In [ ]:
"""## Test"""

First test results

In [ ]:
phase_test = 'test_model_eval'

In [ ]:
get_results(
    model      = model,
    dataset    = test_dataset,
    batch_size = batch_size,
    threshold  = threshold,
    phase      = phase_test
)

In [ ]:
print("First test successfully completed.")

Second test results

In [ ]:
trainer_predict = trainer.predict(test_dataset)
print(f"trainer_predict: {type(trainer_predict)} len={len(trainer_predict)}\n{trainer_predict}")
print(f"trainer_predict.predictions: {type(trainer_predict.predictions)} shape={trainer_predict.predictions.shape}")  # Model logits
print(f"trainer_predict.label_ids: {type(trainer_predict.label_ids)} shape={trainer_predict.label_ids.shape}")        # Ground truth labels
print(f"trainer_predict.metrics: {type(trainer_predict.metrics)} len={len(trainer_predict.metrics)}")

In [ ]:
trainer_predict_json_serializable = {
    'predictions': trainer_predict.predictions.tolist(),  # Convert Numpy array to list
    'label_ids'  : trainer_predict.label_ids.tolist(),    # Convert Numpy array to list
    'metrics'    : trainer_predict.metrics                # Dictionary is already serializable
}

In [ ]:
file_path = "trainer_predict.json"
with open(file_path, "w") as f:
  json.dump(trainer_predict_json_serializable, f)
print(f"Test output successfully saved to {file_path}.")

In [ ]:
print("Second test successfully completed.")

Upload Test Results to HF repo_id_dataset

In [ ]:
upload_file(
    path_or_fileobj = f"{phase_test}_results.csv",
    path_in_repo    = f"{phase_test}_results.csv",
    repo_id         = HF_name,
    repo_type       = 'dataset'
)
upload_file(
    path_or_fileobj = f"{phase_test}_metrics.json",
    path_in_repo    = f"{phase_test}_metrics.json",
    repo_id         = HF_name,
    repo_type       = 'dataset'
)
upload_file(
    path_or_fileobj = 'trainer_predict.json',
    path_in_repo    = 'trainer_predict.json',
    repo_id         = HF_name,
    repo_type       = 'dataset'
)

In [ ]:
print("It's the end")